# 🧠 Módulo 5 — Programación Funcional en Python

Python no es 100% funcional, pero soporta muchas herramientas del paradigma:

- Funciones como ciudadanos de primera clase
- Funciones puras
- Lambdas
- `map`, `filter`, `reduce`
- Higher-order functions
- Closures

Este notebook explica cómo usarlas de forma realista en Python moderno.

---
## 1️⃣ Funciones como "ciudadanos de primera clase"

En Python, las funciones:
- se pueden almacenar en variables
- pasar como parámetros
- devolver desde otras funciones
- guardarse en estructuras


In [1]:
def saludar():
    return "Hola"

f = saludar
f(), f()

('Hola', 'Hola')

---
## 2️⃣ Funciones puras

Una **función pura**:
- Siempre produce el mismo resultado con los mismos parámetros
    - Sin efectos secundarios
    - No modifica variables externas


In [2]:
def pura(x, y):
    return x + y

pura(2,3), pura(2,3)

(5, 5)

Una función **no pura** afecta o depende del estado externo:

In [3]:
contador = 0
def impura(x):
    global contador
    contador += x
    return contador

impura(1), impura(1)

(1, 2)

---
## 3️⃣ Lambdas

Funciones pequeñas y anónimas:

In [17]:
#def cuadrado(x):
    #return x*x
cuadrado = lambda x: x*x
print(cuadrado(7))

multiplicar = lambda x,y: x*y
print(multiplicar(5,7))

serieFactor = lambda x: [i*x for i in range(1,11)]
serieExponente = lambda x: ([(i**x for i in range(1,11))] , [i**x for i in range(1,11)])

print(serieFactor(4))
print(serieExponente(2))

class Ejemplo():
    soypropiedad = lambda self,x: x

e = Ejemplo()

print(e.soypropiedad(1))


49
35
[4, 8, 12, 16, 20, 24, 28, 32, 36, 40]
([<generator object <lambda>.<locals>.<genexpr> at 0x77d91acf7510>], [1, 4, 9, 16, 25, 36, 49, 64, 81, 100])
1


---
## 4️⃣ `map`, `filter`, `reduce`

### ✔️ map: aplicar transformación

In [12]:
nums = [1,2,3,4]
print(nums)
print(list(map(lambda x: x*2, nums)))

inc = lambda x: x+1
dec = lambda x: x-1
print(list(map(inc, nums)))
print(list(map(dec, nums)))

strl = lambda x: str(x)
print(list(map(strl, nums)))

[1, 2, 3, 4]
[2, 4, 6, 8]
[2, 3, 4, 5]
[0, 1, 2, 3]
['1', '2', '3', '4']


### ✔️ filter: filtrar valores

In [ ]:
print(list(filter(lambda x: x%2==0, nums)))

par = lambda x: x%2 == 0
impar = lambda x: x%2 != 0

print(list(filter(impar, nums)))

[2, 4]
[1, 3]


### ✔️ reduce: acumular resultados
Necesita import explícito:

In [32]:
from functools import reduce

nums = [1,2,3,4]

#operacion sumatoria
print("Acc1 sumatorio:", reduce(lambda acc, x: acc+x, nums))

#operacion media
print("Acc2 media:", reduce(lambda acc, x: acc+x, nums, 0) / nums.__len__()) 

#operacion compleja
pares = [("id", 1), ("name", "Ana"), ("age", 30)]
reduc1 = reduce(lambda acc, x:
               (
                   acc.update({x[0]:x[1]})
                   or acc
               ),
               pares, 
               {})
               
print("Acc3 dicc:", dict(reduc1))

# multiplica cada elemento por todos los elementos
nums2 = [1,2,3,4,5,6]
reduc2 = reduce(lambda a, c: 
       [x*c for x in a],
       nums2, nums2)

print("Acc4 iter:", reduc2)

Acc1 sumatorio: 10
Acc2 media: 2.5
Acc3 dicc: {'id': 1, 'name': 'Ana', 'age': 30}
Acc4 iter: [720, 1440, 2160, 2880, 3600, 4320]


---
## 5️⃣ Higher-Order Functions (HOF)

Una **HOF** es una función que:
- recibe otra función como argumento
- o devuelve una función nueva

Ejemplo: creador de multiplicadores:

In [33]:
def multiplicador(n):
    def multiplicar(x):
        return x*n
    return multiplicar

por_3 = multiplicador(3)
por_3(10)

30

In [ ]:
#decoradores - LOGS
def log(fn):
    def wrapper(*args, **kvargs):
        print(f"[LOG] {fn.__name__}")
        return fn(*args, **kvargs)
    return wrapper

def procesar1():
    print("procesando1..")

log(procesar1)()

@log
def procesar2():
    print("procesando2..")

@log
def otrafuncion():
    print("otrafuncionando..")

procesar2()

otrafuncion()

[LOG] procesar1
procesando1..
[LOG] procesar2
procesando2..
[LOG] otrafuncion
otrafuncionando..


In [55]:
#decoradores como cache
def cache(fn):
    value = None
    has_value = False
    def wrapper(*args, **kvargs):
        nonlocal value, has_value
        if has_value:
            print('cacheado')
            return value
        else:
            print('sin cache')
            value = fn(*args, **kvargs)
            has_value = True
            return value
    return wrapper

@cache
def calcular(a,b):
    print("calculando..")
    return a+b

print("Ej1:", calcular(2,3))
print("Ej2:", calcular(3,2))
print("Ej3:", calcular(4,5))

from functools import wraps

def cache2(fn):
    memo = {}
    @wraps(fn) 
    def wrapper(*args, **kvargs):
        key = (args, frozenset(kvargs.items()))
        if key not in memo:
            print('sin cache')
            memo[key] = fn(*args, **kvargs)
            return memo[key]
        else:
            print('cacheado')
            return memo[key]
    return wrapper

@cache2
def calcular2(a,b):
    print("calculando2..")
    return a+b

print("Ej4:", calcular2(2,3))
print("Ej5:", calcular2(3,2))
print("Ej5:", calcular2(2,3))
print("Ej6:", calcular2(4,5))


sin cache
calculando..
Ej1: 5
cacheado
Ej2: 5
cacheado
Ej3: 5
sin cache
calculando2..
Ej4: 5
sin cache
calculando2..
Ej5: 5
cacheado
Ej5: 5
sin cache
calculando2..
Ej6: 9


In [51]:
# usar en pipelines
def pipeline(data, *steps):
    for step in steps:
        data = map(step, data)
    return list(data)

p = pipeline([1,2,3],
    lambda x: x*2, lambda x: x+1,
    )

print(p)

[3, 5, 7]


In [63]:
import time

def timer(fn):
    @wraps(fn)
    def wrapper(*args, **kvargs):
        ini = time.perf_counter()
        res = fn(*args, **kvargs)
        fin = time.perf_counter()
        print(f"{fn.__name__} ejecutada en {(fin - ini) * 1000:3f} ms")
        return res
    return wrapper

@timer
def calcular3():
    suma = 0
    for _ in range(10_000_000):
        suma += 1
    return suma

calcular3()

calcular3 ejecutada en 333.636680 ms


10000000

In [64]:
def route(method, path):
    def decorator(handler):
        api[(method,path)] = handler
        return handler
    return decorator

@route('POST',"/users")
def usuariosHandler(request):
    return

NameError: name 'api' is not defined

---
## 6️⃣ Closures

Una función interna recuerda las variables de su función externa, incluso después de haber terminado.

In [66]:
def contador():
    n = 0
    def inc():
        nonlocal n
        n += 1
        return n
    return inc

c = contador()
o = contador()
c(), c(), c(), o(), o()

(1, 2, 3, 1, 2)

In [82]:
# EJ: CONTROL DE ACCESO
# que solo se pueda llamar a una funcion según el rol
def required_role(role):
    def decorator(fn):
        def wrapper(user, *args):
            if user['role'] != role:
                #raise PermissionError('Acceso denegado')
                print('Acceso denegado para el rol:', user['role'])
            else:
                return fn(user, *args)
        return wrapper
    return decorator

@required_role("admin")
def borrar_usuario(user):
    return print("Usuario:", user, "borrado")

borrar_usuario({'role':'admin', 'name':'pepito'})
borrar_usuario({'role':'reader', 'name':'pepito'})

Usuario: {'role': 'admin', 'name': 'pepito'} borrado
Acceso denegado para el rol: reader


In [ ]:
# EJ: generador de funciones que aceptan un numero y devuelven un numero. 
# Gracias a este closure tenemos un comportamiento igual con cualquier funcion que genere. Me aseguro de que serán homogéneas
def aplicar_estrategia(f):
    def procesar(x):
        return f(x)
    return procesar

doble = aplicar_estrategia(lambda v:v*2)
triple = aplicar_estrategia(lambda v:v*3)

doble(2), triple(2)


(4, 6)

In [ ]:
#Ej: closure para datos complejos - metemos propiedades como si fueran privadas
def usuario(nombre, edad):
    _edad = edad
    
    def incrementar():
        nonlocal _edad
        _edad += 1
        return _edad

    def decrementar():
        nonlocal _edad
        _edad -= 1
        return _edad

    def crea_usuario():
        return {
            "nombre": nombre,
            "get_edad": lambda: _edad,
            "incrementar": incrementar,
            "decrementar": decrementar
        }
    return crea_usuario()
    
# con esto tendremos un usuario como si fuera un objeto
u = usuario('david',30)

print(u["nombre"])
print(u["get_edad"]())
print(u["incrementar"]())
print(u["get_edad"]())
print(u["decrementar"]())
print(u["get_edad"]())

david
30
31
31
30
30


---
## 7️⃣ Ejemplo real: pipeline funcional

Pipeline para limpieza de datos:

In [91]:
datos = ["  Hola  ", "MUNDO", "  Python "]
resultado = list(
    map(lambda t: t.strip().lower(), datos)
)
resultado

['hola', 'mundo', 'python']

In [109]:
#EJ: VALIDADOR DE PARAMETROS (de una funcion)
def validar(regla):
    def check(x):
        if not regla(x):
            raise ValueError('Dato Invalido')
        return x
    return check

# Mis reglas
solo_positivos = validar(lambda x: x > 0)
solo_negativos = validar(lambda x: x < 0)

solo_positivos(5)
#solo_positivos(-1)

# creo un decorador con esta funcion
def validar_args(regla):
    def decorator(fn):
        @wraps(fn)
        def wrapper(*args, **kvargs):
            nargs = [validar(regla)(arg) for arg in args]
            nkvargs = {k:validar(regla)(v) for k,v in kvargs.items()}
            return fn(*nargs, **nkvargs)
        return wrapper
    return decorator

@validar_args(lambda x:x>0)
def multiplicar(a, b):
    return a * b;

print("Ej1:", multiplicar(3,5))
#multiplicar(3,0)

@validar_args(lambda x: isinstance(x,str))
def concatenar(a,b):
    return a+b
print("Ej2:", concatenar('Hola', 'Mundo'))
#print("Ej2_2:", concatenar('Hola', 45))


Ej1: 15
Ej2: HolaMundo


---
## 8️⃣ Ejercicio práctico

### �� Ejercicio
Dado:
```python
numeros = [1,2,3,4,5,6,7,8,9,10]
```
Usa **solo funciones funcionales** (`map`, `filter`, `reduce`) para obtener:

1. Los números pares
2. Su cuadrado
3. La suma final

Escribe tu solución aquí:

In [ ]:
nums = [1,2,3,4,5,6,7,8,9,10]
print("Datos:", nums)

par = lambda x: x%2 == 0
impar = lambda x: x%2 != 0

numsPares = list(filter(par, nums))
print("1. Los numeros pares:", numsPares)

numsCuadrados = list(map(lambda x:x*x, numsPares))
print("2. El cuadrado:", numsCuadrados)

sumaNums = reduce(lambda acc, x: acc+x, numsCuadrados)
print("3. La suma final:", sumaNums)


Datos: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
1. Los numeros pares: [2, 4, 6, 8, 10]
2. El cuadrado: [4, 16, 36, 64, 100]
3. La suma final: 220


---
## ✅ Solución (oculta)

<details>
<summary>Mostrar solución</summary>

```python
from functools import reduce

pares = list(filter(lambda x: x%2==0, numeros))
cuadrados = list(map(lambda x: x*x, pares))
total = reduce(lambda acc,x: acc+x, cuadrados)

pares, cuadrados, total
```
</details>